[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/quick-tour/metadata-filtering.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/quick-tour/metadata-filtering.ipynb)

# Metadata filtering with Pinecone

Metadata filtering is a new feature in Pinecone that allows you to apply filters on vector search based on metadata.
You can add the metadata to the embeddings within Pinecone, and then filter for those criteria when sending the query. Pinecone will search for similar vector embeddings only among those items that match the filter.
The metadata filtering accepts arbitrary filters on metadata, and it retrieves exactly the number of nearest-neighbor results that match the filters. For most cases, the search latency will be even lower than unfiltered searches.

In this notebook, we will walk through a simple use of filtering while performing vector search on documents.

## Prerequisites

Set up Pinecone.

In [ ]:
import os
import lantern_pinecone as pinecone

LANTERN_DB_URL = os.environ.get('LANTERN_DB_URL') or 'postgres://postgres@localhost:5432'
pinecone.init(LANTERN_DB_URL)

## Creating the Index

In [11]:
index_name = "pinecone-metadata-filtering"

In [12]:
import time

# Delete index if exists
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

# Create an index
pinecone.create_index(name=index_name, dimension=2, metric="euclidean")

# wait for index to be ready before connecting
while not pinecone.describe_index(index_name).status['ready']:
    time.sleep(1)

In [13]:
# Connect to the index
index = pinecone.Index(index_name=index_name)

### Generate sample document data

In [14]:
# Generate some data
import pandas as pd

df = pd.DataFrame()
df["id"] = ["F-1", "F-2", "S-1", "S-2"]
df["vector"] = [[1., 1.], [2., 2.], [3., 3.], [4., 4.]]
df["metadata"] = [
    {"category": "finance", "published": 2015},
    {"category": "finance", "published": 2016},
    {"category": "sport", "published": 2017},
    {"category": "sport", "published": 2018}]
df

,id,vector,metadata
0,F-1,"[1.0, 1.0]","{'category': 'finance', 'published': 2015}"
1,F-2,"[2.0, 2.0]","{'category': 'finance', 'published': 2016}"
2,S-1,"[3.0, 3.0]","{'category': 'sport', 'published': 2017}"
3,S-2,"[4.0, 4.0]","{'category': 'sport', 'published': 2018}"


### Insert vectors

In [15]:
# Insert vectors without specifying a namespace
index.upsert(vectors=zip(df.id, df.vector, df.metadata))
index.describe_index_stats()

{'dimensions': 2,
 'index_fullness': 1.0,
 'total_count': 4,
 'namespaces': {'': {'vector_count': 4}}}

### Fetch a vector

In [18]:
index.fetch(ids=["F-1"])

{'namespace': '',
 'vectors': {'F-1': {'id': 'F-1',
   'embedding': [1.0, 1.0],
   'metadata': {'category': 'finance', 'published': 2015},
   'distance': [1.0, 1.0]}}}

### Query top-3 without filtering

In [19]:
query_results = index.query(df[df.id == "F-1"].vector[0], top_k=3)
query_results

{'namespace': '',
 'matches': [{'id': 'F-1', 'score': 0.0, 'values': None, 'metadata': None},
  {'id': 'F-2', 'score': 2.0, 'values': None, 'metadata': None},
  {'id': 'S-1', 'score': 8.0, 'values': None, 'metadata': None}]}

### Query results with articles in finance published after 2015

We should expect to see only 1 article that matches this query.

In [20]:
filter_condition = {
    "category" : {"$eq": "finance"},
    "published": {"$gt": 2015 }
}
query_results = index.query(
    df[df.id == "F-1"].vector[0], top_k=3, filter=filter_condition
)
query_results

{'namespace': '',
 'matches': [{'id': 'F-2', 'score': 2.0, 'values': None, 'metadata': None}]}

### Delete the index

In [21]:
# delete the index
pinecone.delete_index(index_name)